# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
from api_keys import g_key
gmaps.configure(api_key=g_key)
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("weather data.csv")
weather_data.dropna(inplace=True)
weather_data.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,puerto ayora,-0.7393,-90.3518,23.64,91,54,1.01,EC,1673658402
1,ushuaia,-54.8000,-68.3000,9.81,46,40,32.21,AR,1673658403
2,rikitea,-23.1203,-134.9692,25.68,76,4,16.89,PF,1673658404
3,isangel,-19.5500,169.2667,28.17,62,26,11.36,VU,1673658405
4,faanui,-16.4833,-151.7500,26.37,80,100,22.86,PF,1673658406
5,avarua,-21.2078,-159.7750,28.03,54,0,5.75,CK,1673658407
6,sakakah,29.9697,40.2064,7.78,79,0,7.27,SA,1673658409
7,orissaare,58.5592,23.0826,4.98,93,56,15.86,EE,1673658409
8,arraial do cabo,-22.9661,-42.0278,24.25,92,100,22.28,BR,1673658410
9,bluff,-46.6000,168.3333,15.67,64,11,11.27,NZ,1673658411


In [3]:
weather_data.count()

City          547
Lat           547
Lng           547
Max Temp      547
Humidity      547
Cloudiness    547
Wind Speed    547
Country       547
Date          547
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
city_locations = weather_data[["Lat", "Lng"]]

In [5]:
humidity = weather_data["Humidity"]

In [6]:
fig = gmaps.figure()

In [7]:
heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, dissipating=False, max_intensity = 10, point_radius = 1)

In [8]:
fig.add_layer(heat_layer)

In [10]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
hotel_df = weather_data[(weather_data["Max Temp"]<27) & (weather_data["Max Temp"]>21) & (weather_data["Cloudiness"]==0)  & (weather_data["Wind Speed"]<4.5)]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [13]:
hotel_df["Hotel Name"] = ""

C:\Users\nickj\anaconda3\anacondatake2\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
#check df to see if column was created
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
84,santiago del estero,-27.7951,-64.2615,24.34,35,0,1.79,AR,1673658485,
90,pochutla,15.7432,-96.4661,26.01,75,0,4.32,MX,1673658492,
185,hambantota,6.1241,81.1185,21.19,100,0,2.30,LK,1673658465,
196,cabo san lucas,22.8909,-109.9124,24.61,68,0,3.44,MX,1673658572,
317,coahuayana,18.7333,-103.6833,23.28,72,0,1.68,MX,1673658718,
328,moron,-34.6534,-58.6198,26.10,23,0,4.41,AR,1673658730,


In [15]:
params = {"key":g_key,
          "radius": 5000
    
}

In [16]:
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lon = row["Lng"]
    params["location"] = f"{lat}, {lon}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
    try:
        #use index one as 0 contains information on the city, not the hotel name
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][1]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "None nearby"


C:\Users\nickj\anaconda3\anacondatake2\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [17]:
hotel_name["results"][1]["name"]

IndexError: list index out of range

In [18]:
#check to see the type it is stored in
type(hotel_name)

dict

In [19]:
#it is a dictionary, so check to see the keys
hotel_name.keys()

dict_keys(['error_message', 'html_attributions', 'results', 'status'])

In [20]:
hotel_name["results"][1]["name"]

IndexError: list index out of range

In [21]:
#check to see how many hotels there are
len(hotel_name)

4

In [22]:
#use index 0 to get the first result
hotel_name["results"]

[]

In [23]:
hotel_df.head(20)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
84,santiago del estero,-27.7951,-64.2615,24.34,35,0,1.79,AR,1673658485,None nearby
90,pochutla,15.7432,-96.4661,26.01,75,0,4.32,MX,1673658492,None nearby
185,hambantota,6.1241,81.1185,21.19,100,0,2.30,LK,1673658465,None nearby
196,cabo san lucas,22.8909,-109.9124,24.61,68,0,3.44,MX,1673658572,None nearby
317,coahuayana,18.7333,-103.6833,23.28,72,0,1.68,MX,1673658718,None nearby
328,moron,-34.6534,-58.6198,26.10,23,0,4.41,AR,1673658730,None nearby


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [25]:
# Add marker layer ontop of heat map
# fig = gmaps.figure()
# heat_layer = gmaps.heatmap_layer(locations, weights= humidity, dissipating=False, max_intensity=10, point_radius = 1)
# fig.add_layer(heat_layer)
# Display figure
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))